# **Hmara code**

In [ ]:
!pip install flask_ngrok

In [ ]:
import os
from os.path import exists, join, basename, splitext
import cv2
import numpy as np
import glob
from subprocess import Popen, PIPE
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  
# from IPython.display import YouTubeVideo

In [ ]:
app=Flask(__name__)

# os.makedirs('/content/inputs/test/image/')
run_with_ngrok(app)   

def create_video(img_path):
  img_array = []

  img = cv2.imread(img_path)
  height, width, layers = img.shape
  size = (width,height)
  img_array.append(img)

  out = cv2.VideoWriter('/content/project.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
  
  for i in range(len(img_array)):
      out.write(img_array[i])
  out.release()

@app.route('/corr', methods=['POST'])
def corr():
  file = request.files['file']
  file.save("/content/image.jpg")
  print("Image saved")

  # try:
  process = Popen("rm -rf project.mp4", shell=True, stdout=PIPE, stderr=PIPE)
  print("Old Video Deleted")
  
  process = Popen(["rm -rf project.avi"], shell=True, stdout=PIPE, stderr=PIPE)
  print("Old Video 2 Deleted")

  create_video("/content/image.jpg")
  print("Video saved")
  
  process = Popen(["cd openpose && ./build/examples/openpose/openpose.bin --face --hand --video /content/project.mp4 --write_json /content/output_json/ --display 0  --write_video ../project.avi"],  shell=True, stdout=PIPE, stderr=PIPE)
  print("Almost Done")
  
  process = Popen(["ffmpeg -y -loglevel info -i project2.avi project2.mp4"], shell=True, stdout=PIPE, stderr=PIPE)
  print("Done Part-1")

  return jsonify({"status": "success"})

  # except:
  #   print("No image found")

# app.run()

if __name__=="__main__":
    app.run()